In [ ]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
import requests
from requests.compat import quote
from mako.template import Template
from PIL import Image, ImageFile
from openpyxl import load_workbook

import calendar
from io import BytesIO
import re

from modules.wmtools import is_commons_file, get_hash

In [ ]:
commons_site = pb.Site("commons", "commons")

In [ ]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [ ]:
EXCEL_FILE = "diario.madrid.es.xlsx"
wb = load_workbook(os.path.join(cwd, EXCEL_FILE))

In [ ]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Diario de Madrid - ${title}]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Images from Ayuntamiento de Madrid (to classify)]]
[[Category:${month} ${year} in Madrid]]"""

In [ ]:
onlyfiles = [os.path.join(images_directory, f) for f in os.listdir(images_directory) if os.path.isfile(os.path.join(images_directory, f))]
for i, image in enumerate(onlyfiles):
    sha = get_hash(image)
    if is_commons_file (sha) == False:
        print (image)
    else :
        #print (image)
        os.remove(image)

In [ ]:
used_names = []

ws = wb[u'Sheet']
for row in ws.iter_rows():
    image_path = row[0].value
    image_name = image_path.split('\\')[-1]
    # The image does not exist. Skipping
    if os.path.isfile(image_path) == False :
        #print ("*Image name ({0}) deleted from local directory".format(image_name))
        continue
    else :
        print ("** Handling image {}".format(image_name))

    # If the image name is already in commons, find a new name
    if pb.Page(commons_site, image_name, ns=6).exists():
        print ("**Image name ({0}) already used in Commons".format(image_name))
        used_names.append(image_name)

    while True:
        if image_name in used_names :
            # Finding a new name
            image_subject = '.'.join(image_name.split('.')[:-1])
            image_extension = 'jpg'
            p = re.compile('(.*) ([0-9]{2}\.jpg)')
            m = p.match(image_name)
            if m is None:
                image_name = image_subject + ' 01.' + image_extension
            else :
                counter = int(m.group(2)[:2]) + 1
                image_name = '{} {:02d}.{}'.format(m.group(1), counter, image_extension)

            if pb.Page(commons_site, image_name, ns=6).exists():
                print ("Image name ({0}) already used in Commons. Finding a new name".format(image_name))
                used_names.append(image_name)
        else :
            print ("Preparing to upload image with name {0}".format(image_name))
            used_names.append(image_name)
            break

    # image upload
    vars = {
            "url": row[3].value,
            "description": row[7].value,
            "year": row[5].value,
            "month": row[6].value,
            "date": row[4].value,
            "title": row[2].value
            }
    t = Template(template)
    _text = t.render(**vars)

    bot = UploadRobot([image_path],
                      description = _text,
                      useFilename = image_name,
                      keepFilename = True,
                      verifyDescription = False,
                      ignoreWarning = True,
                      targetSite = commons_site)
    bot.run()

    os.remove(image_path)